In [1]:
import numpy as np
import pandas as pd
from openbb import obb
import statsmodels.api as sm
from statsmodels import regression
import matplotlib.pyplot as plt
obb.user.preferences.output_type = "dataframe"

symbols = ["NEM","RGLD","SSRM","CDE","LLY","UNH","JNJ","MRK","SPY"]
data = obb.equity.price.historical(
    symbols,
    start_date="2024-01-01",
    end_date="2025-01-01",
    provider="yfinance"
).pivot(columns="symbol",values="close")

benchmark_returns = data.pop("SPY").pct_change().dropna()
portfolio_returns = data.pct_change().dropna()
portfolio_returns.name = "portfolio"
X = benchmark_returns.values
def linreg(x,y):
    x = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x).fit()
    return model.params[0], model.params[1]
betas = {}
alphas = {}
for col in portfolio_returns.columns:
    y = portfolio_returns[col].values
    alphas[col], betas[col] = linreg(X, y)   # slope vs SPY (beta)

betas = pd.Series(betas, name="beta").sort_values()
alphas = pd.Series(alphas, name="alpha")
print(betas)
print(alphas)


c:\Users\nickt\anaconda3\envs\my-quant-stack\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


JNJ    -0.040037
UNH     0.127909
MRK     0.219790
NEM     0.674669
RGLD    0.772632
LLY     0.871589
SSRM    1.677509
CDE     2.285955
Name: beta, dtype: float64
CDE     0.001231
JNJ    -0.000321
LLY     0.000463
MRK    -0.000624
NEM    -0.000706
RGLD   -0.000159
SSRM   -0.001715
UNH    -0.000221
Name: alpha, dtype: float64
